# 1. 라이브러리 설치하기

In [1]:
#라이브러리 호출
import cv2
import os

print(cv2.__version__)


4.5.5


In [2]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

In [3]:
!pip install pillow

# 2. Azure Cognitive Service 이미지분석 불러오기

### ※ 참고사항!

#### 1) URL 기반으로 Description 만들 때 : Describe_image 메써드 사용
- Azure Cognitive Services Computer Vision SDK for Python : 이게 BEST
https://docs.microsoft.com/ko-kr/python/api/overview/azure/cognitiveservices-vision-computervision-readme?view=azure-python

- <참고만> : 오류뜸 https://docs.microsoft.com/ko-kr/azure/cognitive-services/computer-vision/quickstarts-sdk/image-analysis-client-library?tabs=visual-studio&pivots=programming-language-python

- <참고만> : 오류뜸 https://docs.microsoft.com/ko-kr/azure/cognitive-services/computer-vision/vision-api-how-to-topics/howtocallvisionapi?tabs=python


#### 2) 로컬 경로에서 Description 만들 때 : describe_image_in_stream 메써드 사용
https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/ComputerVision/ImageAnalysisQuickstart.py

### 1. 라이브러리 불러오기

In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

### 2. 계정 인증하기

In [5]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''

#key와 엔드포인트는 azure portal에서 cognitive service를 만들고 나면 가져올 수 있음
#현재의 키와 엔드포인트는 내 계정인데, 다른 컴퓨터에서도 돌아갈지는 모르겠음

subscription_key = "9e14cf501b8b4ca98a27c3bb2391e803"
endpoint = "https://lmg1111111.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

### 3. 이미지 가져오기

In [6]:
# Images used for the examples: Describe an image, Categorize an image, Tag an image, 
# Detect faces, Detect adult or racy content, Detect the color scheme, 
# Detect domain-specific content, Detect image types, Detect objects

#images가 들어간 폴더를 찾는다.
images_folder = os.path.join (os.path.dirname(os.path.abspath('')), "images")
# 아래의 url은 의미없다.
remote_image_url = "https://moderatorsampleimages.blob.core.windows.net/samples/sample16.png"

'''
END - Quickstart variables
'''

'\nEND - Quickstart variables\n'

### 4. 분석하기

In [7]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.
'''
print("===== Describe an Image - local =====")
# images 폴더에서 image2.png 파일을 불러온다
#혹시나 해서 실험해 봤는데 png든 jpg든 결과에는 관계없다. 확장자에 따라 결과가 달라지지 않는다
local_image_path = os.path.join(images_folder, "image2.png")
local_image = open(local_image_path, "rb")

# Call API
description_result = computervision_client.describe_image_in_stream(local_image)

# Get the captions (descriptions) from the response, with confidence level
print("Description of local image: ")
if (len(description_result.captions) == 0):
    print("No description detected.")
else:
    for caption in description_result.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
print()
'''
END - Describe an Image - local
'''

===== Describe an Image - local =====
Description of local image: 
'a person and a boy looking at a screen' with confidence 37.11%



'\nEND - Describe an Image - local\n'

### 5. 분석결과만 따로 뽑아쓰려면?

In [8]:
#text만 따로 뽑아쓰려면 caption[0].text로 뽑아쓸 수 있다
print(description_result.captions[0].text)

a person and a boy looking at a screen


# 4. TTS 가즈아

### ※ 참고사항!

#### 1. 텍스트 음성 변환
https://docs.microsoft.com/ko-kr/azure/cognitive-services/speech-service/get-started-text-to-speech?tabs=terminal&pivots=programming-language-python

#### 2. 텍스트 파일 저장
https://docs.microsoft.com/ko-kr/azure/cognitive-services/speech-service/how-to-speech-synthesis?tabs=script%2Cbrowserjs%2Cterminal&pivots=programming-language-python

### 1. 라이브러리 설치

In [9]:
pip install azure-cognitiveservices-speech

Note: you may need to restart the kernel to use updated packages.


### 2. 스피커로 분석결과를 읽어보자

In [11]:
import azure.cognitiveservices.speech as speechsdk


# 여기서도 key와 region은 portal에서 cognitive service를 만들면 뽑을 수 있다
speech_config = speechsdk.SpeechConfig(subscription="4273fb5b821a4785bc85836bea53d2a1", region="KoreaCentral")
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# 제니씨의 음성으로 들어보자
speech_config.speech_synthesis_voice_name='en-US-JennyNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

# Get text from the console and synthesize to the default speaker.
print("Enter some text that you want to speak >")

#text = input()을 넣으면 따로 내가 출력하고 싶은 글귀를 넣을 수도 있다
#일단은 아까의 분석결과를 넣어보도록 하자
text = description_result.captions[0].text

speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

#스피커로 출력하기
if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = speech_synthesis_result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

Enter some text that you want to speak >
Speech synthesized for text [a person and a boy looking at a screen]


### 3. 파일로 저장해보자

In [37]:
#filename에는 저장할 폴더와 저장할 파일명을 넣으면 됩니다.
audio_config = speechsdk.audio.AudioOutputConfig(filename="C:/Users/ansrl/Documents/이문기/11_TAVE/2022 AI_SPARK/jeiyoon.wav")
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
synthesizer.speak_text_async(description_result.captions[0].text)